## Calling SNPs 

### Using samtools mpileup 

#### First I need to convert my SAM alignment files to BAM files. Then sort and index it

In [10]:
!pwd

/Users/laurelnave-powers/Desktop/GitHub/Laurel-genes/data


In [12]:
cd /Applications/bioinfo/samtools1.11/bin

/Applications/bioinfo/samtools1.11/bin


In [14]:
! ls

ace2sam               md5fa                 samtools
blast2sam.pl          md5sum-lite           samtools.pl
bowtie2sam.pl         novo2sam.pl           seq_cache_populate.pl
export2sam.pl         plot-ampliconstats    soap2sam.pl
interpolate_sam.pl    plot-bamstats         wgsim
maq2sam-long          psl2sam.pl            wgsim_eval.pl
maq2sam-short         sam2vcf.pl            zoom2sam.pl


In [2]:
!samtools


Program: samtools (Tools for alignments in the SAM format)
Version: 1.11 (using htslib 1.11)

Usage:   samtools <command> [options]

Commands:
  -- Indexing
     dict           create a sequence dictionary file
     faidx          index/extract FASTA
     fqidx          index/extract FASTQ
     index          index alignment

  -- Editing
     calmd          recalculate MD/NM tags and '=' bases
     fixmate        fix mate information
     reheader       replace BAM header
     targetcut      cut fosmid regions (for fosmid pool only)
     addreplacerg   adds or replaces RG tags
     markdup        mark duplicates
     ampliconclip   clip oligos from the end of reads

  -- File operations
     collate        shuffle and group alignments by name
     cat            concatenate BAMs
     merge          merge sorted alignments
     mpileup        multi-way pileup
     sort           sort alignment file
     split          splits a file by read group
     quickcheck     quickly check if SA

In [18]:
%%bash
# Convert SAM to sorted BAM
samtools view -bS ../analyses/HIPP-DR.sam \
| samtools sort -o ../analyses/HIPP-DR.sorted.bam -

# Create BAM index file
samtools index ../analyses/HIPP-DR.sorted.bam

[bam_sort_core] merging from 13 files and 1 in-memory blocks...


#### Then I need to mpileup it

In [28]:
!pwd

/Users/laurelnave-powers/Desktop/GitHub/Laurel-genes/code


In [26]:
!gunzip -k ../data/Danio_rerio.GRCz11.dna_sm.toplevel.fa.gz

In [29]:
%%bash
samtools mpileup --VCF --no-BAQ \
--fasta-ref ../data/Danio_rerio.GRCz11.dna_sm.toplevel.fa \
../analyses/HIPP-DR.sorted.bam \
-o ../analyses/HIPP-DR.vcf.gz

[warning] samtools mpileup option `v` is functional, but deprecated. Please switch to using bcftools mpileup in future.
[mpileup] 1 samples in 1 input files


#### Now repeat with MICPACI

In [30]:
%%bash
# Convert SAM to sorted BAM
samtools view -bS ../analyses/MICPACI-DR.sam \
| samtools sort -o ../analyses/MICPACI-DR.sorted.bam -

# Create BAM index file
samtools index ../analyses/MICPACI-DR.sorted.bam

[bam_sort_core] merging from 14 files and 1 in-memory blocks...


In [31]:
%%bash
samtools mpileup --VCF --no-BAQ \
--fasta-ref ../data/Danio_rerio.GRCz11.dna_sm.toplevel.fa \
../analyses/MICPACI-DR.sorted.bam \
-o ../analyses/MICPACI-DR.vcf.gz

[warning] samtools mpileup option `v` is functional, but deprecated. Please switch to using bcftools mpileup in future.
[mpileup] 1 samples in 1 input files


### Now I have .vcf.gz files for both of my species. Next I will use bcfcalls to call certain sites as variant or not.

In [3]:
!bcftools call -v -m ../analyses/HIPP-DR.vcf.gz > ../analyses/HIPP-DR_calls.vcf.gz

Note: none of --samples-file, --ploidy or --ploidy-file given, assuming all sites are diploid


In [4]:
!bcftools call -v -m ../analyses/MICPACI-DR.vcf.gz > ../analyses/MICPACI_calls.vcf.gz

Note: none of --samples-file, --ploidy or --ploidy-file given, assuming all sites are diploid


In [5]:
!zgrep "^##" -v ../analyses/HIPP-DR_calls.vcf.gz | \
awk 'BEGIN{OFS="\t"} {split($8, a, ";"); print $1,$2,$4,$5,$6,a[1],$9,$10}'

#CHROM	POS	REF	ALT	QUAL	INFO	FORMAT	../analyses/HIPP-DR.sorted.bam
6	26832552	A	G	7.30814	DP=1	GT:PL	1/1:36,3,0
6	26832565	A	G	7.30814	DP=1	GT:PL	1/1:36,3,0
6	26832646	C	G	7.30814	DP=1	GT:PL	1/1:36,3,0
6	26832648	C	T	4.38466	DP=1	GT:PL	1/1:32,3,0
6	26832669	C	G	7.30814	DP=1	GT:PL	1/1:36,3,0
6	26832738	T	C	7.30814	DP=1	GT:PL	1/1:36,3,0
6	26832739	C	G	7.30814	DP=1	GT:PL	1/1:36,3,0
17	53337855	C	G	5.06224	DP=7996	GT:PL	1/1:30,255,0
17	53337874	A	G	3.78137	DP=8001	GT:PL	1/1:31,255,0
17	53337876	C	G	5.06187	DP=8001	GT:PL	1/1:30,255,0
17	53337891	G	C	5.06314	DP=8000	GT:PL	1/1:30,255,0
17	53337892	G	C	3.78413	DP=8000	GT:PL	1/1:31,255,0
17	53337894	G	C	5.04505	DP=8000	GT:PL	1/1:30,255,0
17	53337897	A	C	5.76949	DP=7956	GT:PL	1/1:31,255,0
17	53337900	A	C	5.76672	DP=7856	GT:PL	1/1:31,255,0
17	53337909	G	T	5.77394	DP=8000	GT:PL	1/1:31,255,0
17	53337911	C	A	5.05379	DP=7999	GT:PL	1/1:30,255,0
17	53337918	G	A	5.76917	DP=8000	GT:PL	1/1:31,255,0
17	53337921	C	G	5.77355	DP=7999	GT:PL	1/1:31,255,0
17	533

In [7]:
!zgrep "^##" -v ../analyses/MICPACI_calls.vcf.gz | \
awk 'BEGIN{OFS="\t"} {split($8, a, ";"); print $1,$2,$4,$5,$6,a[1],$9,$10}'

#CHROM	POS	REF	ALT	QUAL	INFO	FORMAT	../analyses/MICPACI-DR.sorted.bam
17	53338219	T	A	3.22451	DP=22	GT:PL	1/1:30,66,0
17	53338241	C	T	3.22451	DP=22	GT:PL	1/1:30,66,0
